# Integrated Hybrid: Blending classical and quantum computing
In this sample, you will discover how to blend classical and quantum instructions in the same program, all fully processed by the quantum computing back-end.
You can learn more how to program with Integrated hybrid at https://aka.ms/AQ/Hybrid/Docs

This program will examine a 3 qubit Greenberger–Horne–Zeilinger (GHZ) state and counts the number of times it sees the entanglement fail out of 10 attempts. Unlike in perfect, noiseless simulations, entanglement failures can occur in real-world runs, which are subject to the effects of noise.

In the CheckGHZ() operation, you see three capabilities of Integrated Hybrid being demonstrated:
1. **Branching**: based on measurement results you can adjust the classical code being performed
```Q#
   if not (r0 == r1 and r1 == r2)
```
2. **Classical comput**e: here an arithmetic operation is performed
```Q#
   set mismatch += 1;
```
3. **While the physical qubits are still coherent**, #1 and #2 are performed 




In [ ]:
%azure.connect "/subscriptions/916dfd6d-030c-4bd9-b579-7bb6d1926e97/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/L3-BugBash-2" location="eastus2euap"



## Define the CheckGHZ operation
Please note that you will see a warning displayed (QS5026). It can be ignored. It will be addressed in a subsequent release of the Quantum Development Kit.

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation CheckGHZ() : Int {
    use q = Qubit[3];
    mutable mismatch = 0;
    for _ in 1..10 {
        H(q[0]);
        CNOT(q[0], q[1]);
        CNOT(q[1], q[2]);

        // Measures and resets the 3 qubits
        let (r0, r1, r2) = (MResetZ(q[0]), MResetZ(q[1]), MResetZ(q[2]));

        // Adjusts classical value based on measurement results
        if not (r0 == r1 and r1 == r2) {
            set mismatch += 1;
        }
    }
    return mismatch;
}


## Submit the operation
For 50 shots, as configured below, the following will use about 10.65 units (HQCs or eHQCs).<br/>
If the job is not completed after the timeout (300 seconds), you can query the status followed by querying the output.<br/>
You can also check the status under Operations > Job management.

**Understanding the results**<br/>
The histogram reports the ratio, out of the number of shots performed, that a given number of entanglements failed.<br/>
The histogram values can range from 0 (no entanglement failed) to 10 (all entanglements failed).

In [ ]:
%azure.target quantinuum.sim.h1-1e
%azure.target-capability AdaptiveExecution
%azure.execute CheckGHZ shots=50 timeout=300 poll=10


In [ ]:
%azure.status


In [ ]:
%azure.output


## ↗ Learning more about Integrated hybrid
Now that you have run this sample, you can further learn and experiment with the _Error Correction_ and the _Iterative Phase Estimation_ samples also available in the gallery.

To learn more, please refer to the [online documentation](https://aka.ms/AQ/Hybrid/Docs).